  - Create a model from a local file
  - Create a model from an mlflow model
  - Create a model from cloud path
  - Create a model from a run
  - Read/write model in a job

## Connect to Azure Machine Learning Workspace

In [ ]:
# Import the required libraries

%pip install azure.ai.ml --extra-index-url  https://azuremlsdktestpypi.azureedge.net/sdk-cli-v2

# import required libraries
from azure.ai.ml import MLClient
from azure.ai.ml.entities import Model
from azure.identity import DefaultAzureCredential
from azure.ai.ml.constants import AssetTypes

## Configure workspace details and get a handle to the workspace

In [ ]:
# Enter details of your AML workspace
subscription_id = "<SUBSCRIPTION_ID>"
resource_group = "<RESOURCE_GROUP>"
workspace = "<AML_WORKSPACE_NAME>"

In [ ]:
from azure.ai.ml import MLClient
from azure.identity import DefaultAzureCredential

# get a handle to the workspace
ml_client = MLClient(
    DefaultAzureCredential(), subscription_id, resource_group, workspace
)

## Create a model

- Azure ML models consist of the binary file(s) that represent a machine learning model and any corresponding metadata. Models can be created from a local file or directory. The created model will be tracked in the workspace under the specified name and version.

- The Model class can be used to create a model. It accepts the following key parameters:

  - `name` - Name of the model.

  - `version` - Version of the model. If omitted, Azure ML will autogenerate a version.

  - `path` - Local path to the model file(s). This can point to either a file or a directory.

  - `type` - Storage format of the model. Applicable for no-code deployment scenarios. Allowed values are custom_model, mlflow_model, triton_model

  - `description` - Description of the model.

# Create a model from a local file

- Create a model from a local pkl file and specify the type of the model to be custom_model. The model is initialized with the required parameters. It is then created in the Azure Machine Learning workspace using the MLClient.

In [ ]:
from azure.ai.ml.entities import Model
from azure.ai.ml.constants import AssetTypes

file_model = Model(
    path="mlflow-model/model.pkl",
    type=AssetTypes.CUSTOM_MODEL,
    name="local-file-example",
    description="Model created from local file.",
)
ml_client.models.create_or_update(file_model)

## Create a model from an mlflow model

- Create a model from a local folder, specifying the type of the model to be mlflow_model. The model is initialized with the required parameters. It is then created in the Azure Machine Learning workspace using the MLClient.

In [ ]:
from azure.ai.ml.entities import Model
from azure.ai.ml.constants import AssetTypes

mlflow_model = Model(
    path="mlflow-model",
    type=AssetTypes.MLFLOW_MODEL,
    name="local-mlflow-example",
    description="MLflow model created from local path",
)
ml_client.create_or_update(mlflow_model)

## Create a model from cloud path

-  Create a model from a cloud path, specifying the format of the model to be custom_model. The model is initialized with the required parameters. It is then created in the Azure Machine Learning workspace using the MLClient.

In [ ]:
from azure.ai.ml.entities import Model
from azure.ai.ml.constants import AssetTypes

cloud_model = Model(
    path=file_model.path,
    # The above line basically provides a path in the format "azureml://subscriptions/XXXXXXXXXXXXXXXX/resourceGroups/XXXXXXXXXXX/workspaces/XXXXXXXXXXX/datastores/workspaceblobstore/paths/model.pkl"
    # Users could also use,"azureml://datastores/workspaceblobstore/paths/model.pkl" as a shorthand to the same location
    name="cloud-path-example",
    type=AssetTypes.CUSTOM_MODEL,
    description="Model created from cloud path.",
)
ml_client.models.create_or_update(cloud_model)

##  Create a model from a run

- Create a model from a run, specifying the type of the model to be mlflow_model. The model is initialized with the required parameters. It is then created in the Azure Machine Learning workspace using the MLClient.

In [ ]:
from azure.ai.ml.entities import Model
from azure.ai.ml.constants import AssetTypes

job_name = "<JOB_NAME>"

run_model = Model(
    path=f"azureml://jobs/{job_name}/outputs/artifacts/paths/model/",
    name="run-model-example",
    description="Model created from run.",
    type=AssetTypes.MLFLOW_MODEL,
)
# Uncomment after adding required details above
# ml_client.models.create_or_update(run_model)

## Loading model as an input to a job

- Using the MLflow model trained on the Iris Dataset - `(./mlflow-model) ` and set-up a command that executes the following python code, that loads the model, test data and scores the model:

In [ ]:
with open(args.input_data) as f:
   sample_data = json.load(f)

#columns = ["sepal_length","sepal_width","petal_length","petal_width"]

sk_model = mlflow.sklearn.load_model(args.input_model)
predictions = sk_model.predict(sample_data["data"])

In [ ]:
from azure.ai.ml import command
from azure.ai.ml.entities import Model
from azure.ai.ml.constants import AssetTypes
from azure.ai.ml import Input, Output

# === Note on path ===
# can be can be a local path or a cloud path. AzureML supports `runs:/` and `azureml://` URIs.
# Local paths are automatically uploaded to the default datastore in the cloud.
# More details on supported paths: https://learn.microsoft.com/en-us/azure/machine-learning/how-to-manage-models?tabs=use-local#supported-paths

inputs = {
    "input_data": Input(
        type=AssetTypes.URI_FILE, path="./mlflow-model/input_example.json"
    ),
    "input_model": Input(type=AssetTypes.MLFLOW_MODEL, path="./mlflow-model"),
}

outputs = {
    "output_folder": Output(
        type=AssetTypes.URI_FOLDER,
        path=f"azureml://subscriptions/{subscription_id}/resourcegroups/{resource_group}/workspaces/{workspace}/datastores/workspaceblobstore/paths/predictions",
    )
}

job = command(
    code="./src",  # local path where the code is stored
    command="python load_score.py --input_model ${{inputs.input_model}} --input_data ${{inputs.input_data}} --output_folder ${{outputs.output_folder}}",
    inputs=inputs,
    outputs=outputs,
    environment="AzureML-sklearn-1.0-ubuntu20.04-py38-cpu:1",
    compute="cpu-cluster",
)

# submit the command
returned_job = ml_client.jobs.create_or_update(job)
# get a URL for the status of the job
returned_job.studio_url

## Reading and writing model in a job

- You cannot write to Inputs only Outputs. The code below creates an Output that will mount your AzureML default datastore `(Azure Blob)` in Read-Write mode. The python code simply loads the local mlflow model as input and exports the same model as an output of the job saved in the mounted datastore.

```python
sk_model = mlflow.sklearn.load_model(args.input_model)
mlflow.sklearn.save_model(sk_model, args.output_folder)
```

In [ ]:
from azure.ai.ml import command
from azure.ai.ml.entities import Model
from azure.ai.ml import Input, Output
from azure.ai.ml.constants import AssetTypes

inputs = {"input_model": Input(type=AssetTypes.MLFLOW_MODEL, path="./mlflow-model")}

outputs = {"custom_model_output": Output(type=AssetTypes.CUSTOM_MODEL)}

job = command(
    code="./src",  # local path where the code is stored
    command="python load_write_model.py --input_model ${{inputs.input_model}} --custom_model_output ${{outputs.custom_model_output}}",
    inputs=inputs,
    outputs=outputs,
    environment="AzureML-sklearn-1.0-ubuntu20.04-py38-cpu:1",
    compute="cpu-cluster",
)

# submit the command
returned_job = ml_client.create_or_update(job)
# get a URL for the status of the job
returned_job.studio_url